## Yandex Тренировки по ML2.0 word2vec на PyTorch

Как вы уже могли заметить, идея, лежащая в основе [word2vec](https://arxiv.org/pdf/1310.4546), достаточно общая. В данном задании вы реализуете его самостоятельно.

Дисклеймер: не стоит удивляться тому, что реализация от `gensim` (или аналоги) обучается быстрее и работает точнее. Она использует множество доработок и ускорений, а также достаточно эффективный код. Ваша задача добиться промежуточных результатов за разумное время.

P.s. Как ни странно, GPU в этом задании нам не потребуется.

__Requirements:__ if you're running locally, in the selected environment run the following command:

```pip install --upgrade nltk bokeh umap-learn```


In [ ]:
!pip install --upgrade nltk bokeh umap-learn

In [2]:
import itertools
import random
import string
from collections import Counter
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import umap
from IPython.display import clear_output
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm.auto import tqdm as tqdma

In [3]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

File ‘./quora.txt’ already there; not retrieving.


In [4]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

Токенизация – первый шаг.
Тексты, с которыми мы работаем, включают в себя пунктуацию, смайлики и прочие нестандартные токены, так что простой `str.split` не подойдет.

Обратимся к `nltk` - библиотеку, нашла широкое применеие в области NLP.

In [5]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [6]:
data_tok = [
    tokenizer.tokenize(
        line.translate(str.maketrans("", "", string.punctuation)).lower()
    )
    for line in data
] # генератор в котором токенизируем каждое предложение
data_tok = [x for x in data_tok if len(x) >= 3]

Несколько проверок:

In [7]:
assert all(
    isinstance(row, (list, tuple)) for row in data_tok
), "please convert each line into a list of tokens (strings)"
assert all(
    all(isinstance(tok, str) for tok in row) for row in data_tok
), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all("a" <= x.lower() <= "z" for x in tok)
assert all(
    map(lambda l: not is_latin(l) or l.islower(), map(" ".join, data_tok))
), "please make sure to lowercase the data"

Ниже заданы константы ширины окна контекста и проведена предобработка для построения skip-gram модели.

In [8]:
min_count = 5
window_radius = 5

In [9]:
vocabulary_with_counter = Counter(chain.from_iterable(data_tok))

word_count_dict = dict()
for word, counter in vocabulary_with_counter.items():
    if counter >= min_count:
        word_count_dict[word] = counter

vocabulary = set(word_count_dict.keys())
del vocabulary_with_counter

In [10]:
word_to_index = {word: index for index, word in enumerate(vocabulary)} # (слово, индекс)
index_to_word = {index: word for word, index in word_to_index.items()} # (индекс, слово)

Пары `(слово, контекст)` на основе доступного датасета сгенерированы ниже.

In [11]:
context_pairs = []

for text in data_tok:
    for i, central_word in enumerate(text): # выбираем центральное слово
        context_indices = range(
            max(0, i - window_radius), min(i + window_radius, len(text))
        )
        for j in context_indices:
            if j == i:
                continue
            context_word = text[j]
            if central_word in vocabulary and context_word in vocabulary:
                context_pairs.append(
                    (word_to_index[central_word], word_to_index[context_word]) # нашли пары разрешенных слов и добавили в массив
                )

print(f"Generated {len(context_pairs)} pairs of target and context words.")

Generated 40220313 pairs of target and context words.


#### Подзадача №1: subsampling
Для того, чтобы сгладить разницу в частоте встречаемсости слов, необходимо реализовать механизм subsampling'а.
Для этого вам необходимо реализовать функцию ниже.

Вероятность **оставить** слово из обучения (на фиксированном шаге) вычисляется как
$$
P_\text{save}(w_i)=\sqrt{\frac{t}{f(w_i)}},
$$
где $f(w_i)$ – нормированная частота встречаемости слова, а $t$ – заданный порог (threshold).

In [12]:
def subsample_frequent_words(word_count_dict, threshold=1e-5):
    """
    Calculates the subsampling probabilities for words based on their frequencies.

    This function is used to determine the probability of keeping a word in the dataset
    when subsampling frequent words. The method used is inspired by the subsampling approach
    in Word2Vec, where each word's frequency affects its probability of being kept.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.
    - threshold (float, optional): A threshold parameter used to adjust the frequency of word subsampling.
                                   Defaults to 1e-5.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of keeping each word.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> subsample_frequent_words(word_counts)
    {'the': 0.028, 'is': 0.223, 'apple': 1.0}
    """
    
    ### YOUR CODE HERE
    # import numpy as np
    
    all_w_count = sum(word_count_dict.values())
    freq = {word: word_count_dict[word] / all_w_count for word in word_count_dict}
    prob = {word: (threshold / freq[word]) ** 0.5 for word in freq}
    return prob


In [13]:
word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
subsample_frequent_words(word_counts) # тут тест в шапке функции неправильный эта функция робит, так что все по кайфу

{'the': 0.0034785054261852176,
 'is': 0.007778174593052023,
 'apple': 0.034785054261852175}

#### Подзадача №2: negative sampling
Для более эффективного обучения необходимо не только предсказывать высокие вероятности для слов из контекста, но и предсказывать низкие для слов, не встреченных в контексте. Для этого вам необходимо вычислить вероятност использовать слово в качестве negative sample, реализовав функцию ниже.

В оригинальной статье предлагается оценивать вероятность слов выступать в качестве negative sample согласно распределению $P_n(w)$
$$
P_n(w) = \frac{U(w)^{3/4}}{Z},
$$

где $U(w)$ распределение слов по частоте (или, как его еще называют, по униграммам), а $Z$ – нормировочная константа, чтобы общая мера была равна $1$.

In [14]:
def get_negative_sampling_prob(word_count_dict):
    """
    Calculates the negative sampling probabilities for words based on their frequencies.

    This function adjusts the frequency of each word raised to the power of 0.75, which is
    commonly used in algorithms like Word2Vec to moderate the influence of very frequent words.
    It then normalizes these adjusted frequencies to ensure they sum to 1, forming a probability
    distribution used for negative sampling.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of selecting each word
            for negative sampling.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> get_negative_sampling_prob(word_counts)
    {'the': 0.298, 'is': 0.160, 'apple': 0.042}
    """
    all_w_count = sum(word_count_dict.values())
    freq = {word: (word_count_dict[word] / all_w_count) ** 0.75 for word in word_count_dict}
    Z = sum(freq.values())
    return {word: freq[word] / Z for word in freq}
    
    ### YOUR CODE HERE
    # negative_sampling_prob_dict = {
        # word: 0 for word in word_count_dict.keys()
    # }  # THIS IS A PLACEHOLDER!
    # return negative_sampling_prob_dict

In [15]:
word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
get_negative_sampling_prob(word_counts)

{'the': 0.751488398196177,
 'is': 0.22474745206890812,
 'apple': 0.023764149734914898}

Для удобства, преобразуем полученные словари в массивы (т.к. все слова все равно уже пронумерованы).

In [16]:
keep_prob_dict = subsample_frequent_words(word_count_dict)
assert keep_prob_dict.keys() == word_count_dict.keys()

In [17]:
negative_sampling_prob_dict = get_negative_sampling_prob(word_count_dict)
assert negative_sampling_prob_dict.keys() == negative_sampling_prob_dict.keys()
assert np.allclose(sum(negative_sampling_prob_dict.values()), 1)

In [18]:
# полученные массивы
keep_prob_array = np.array(
    [keep_prob_dict[index_to_word[idx]] for idx in range(len(word_to_index))]
)
negative_sampling_prob_array = np.array(
    [
        negative_sampling_prob_dict[index_to_word[idx]]
        for idx in range(len(word_to_index))
    ]
)

Если все прошло успешно, функция ниже поможет вам с генерацией подвыборок (батчей).

In [19]:
def generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
):
    batch = []
    neg_samples = []

    while len(batch) < batch_size:
        center, context = random.choice(context_pairs)
        if random.random() < keep_prob_array[center]:
            batch.append((center, context))
            neg_sample = np.random.choice(
                range(len(negative_sampling_prob_array)),
                size=num_negatives,
                p=negative_sampling_prob_array,
            )
            neg_samples.append(neg_sample)
    batch = np.array(batch)
    neg_samples = np.vstack(neg_samples)
    return batch, neg_samples

In [20]:
batch_size = 4
num_negatives = 15
batch, neg_samples = generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
)

In [98]:
batch, neg_samples

(array([[26554,  1929],
        [17428, 23546],
        [14156, 26706],
        [ 7272,  2146]]),
 array([[13862, 21398, 17811,  4032, 11866, 25279,  4613, 19383, 26416,
         24813, 14156, 27826, 22595,  6670,  9476],
        [21217,  3643, 18619, 11266,  9096,  9788, 11866,  1741,  2435,
          7025, 10454, 18718, 26558, 27125, 21927],
        [ 1273,  5840, 25295,   757,  6985, 21044, 21565, 26576, 27820,
         10352, 22657,  2930,  1929, 25234,  6740],
        [25772,  1731, 27782,  5112, 10486, 12354, 26052, 15546, 12717,
         12074,  9796, 15287,  4499,  5584, 21501]]))

In [99]:
print(*[(index_to_word[ind1], index_to_word[ind2]) for ind1, ind2 in batch]) 
# видим, что генерируется 4 батча (контесктных пар) и 15 neg sampling примеров

('tweets', 'to') ('just', 'i') ('paid', 'are') ('thicker', 'mashed')


In [100]:
print(*[[index_to_word[ind] for ind in sampl] for sampl in neg_samples], sep='\n') 

['summer', 'determined', 'does', 'profile', 'any', 'when', 'virgin', 'competitive', 'which', 'filmed', 'paid', 'capable', 'earth', 'obvious', 'revenue']
['podcasts', 'why', 'whale', 'digital', 'helplessness', 'from', 'any', 'whats', 'in', 'fbi', 'blurred', 'alcohol', 'would', 'could', 'preferences']
['see', 'ramsey', 'dutch', 'around', 'jamaica', 'intolerance', 'donate', 'speed', 'tell', 'mantle', 'settling', 'bba', 'to', 'internet', 'rest']
['bells', 'lose', 'dont', 'song', 'love', 'money', 'lady', 'it', 'franchising', 'nitie', 'jakku', 'old', 'war', 'sin', 'ash']


Наконец, время реализовать модель. Обращаем ваше внимание, использование линейных слоев (`nn.Linear`) далеко не всегда оправданно!

Напомним, что в случае negative sampling решается задача максимизации следующего функционала:

$$
\mathcal{L} = \log \sigma({\mathbf{v}'_{w_O}}^\top \mathbf{v}_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} \left[ \log \sigma({-\mathbf{v}'_{w_i}}^\top \mathbf{v}_{w_I}) \right],
$$

где:
- $\mathbf{v}_{w_I}$ – вектор центрального слова $w_I$,
- $\mathbf{v}'_{w_O}$ – вектор слова из контекста $w_O$,
- $k$ – число negative samples,
- $P_n(w)$ – распределение negative samples, заданное выше,
- $\sigma$ – сигмоида.

В модели требуется лишь посчитать значения в логорифмах, все остальное будет сделано внутри подсчета лосса

In [356]:
class SkipGramModelWithNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramModelWithNegSampling, self).__init__()
        self.embeddings_in = nn.Embedding(vocab_size, embedding_dim) # center
        self.embeddings_out = nn.Embedding(vocab_size, embedding_dim) # context
        # никакая логсигмоида нам не нужна! это все заложено в лоссе
    
    def forward(self, center_words, pos_context_words, neg_context_words):
        # center_words - входные слова
        # pos_context_words - таргет, т.е. правильный контекст (реально существующий для входного слова)
        # слова которые ТОЧНО не будут в контексте
        # YOUR CODE HERE

        v_in = self.embeddings_in(center_words) 
        v_out = self.embeddings_out(pos_context_words)
        v_neg = self.embeddings_out(neg_context_words)
        
        pos_scores = (torch.sum(v_in * v_out, dim=1))
        neg_scores = (-torch.bmm(v_neg, v_in.unsqueeze(2)).squeeze(2)) #.sum(1) # bmm - батчевое (по 2D-матричное) перемножение матриц
        return pos_scores, neg_scores

In [357]:
device = torch.device("cpu") # TODO: реализовать перенос на cuda

In [358]:
vocab_size = len(word_to_index)
embedding_dim = 32
num_negatives = 15

model = SkipGramModelWithNegSampling(vocab_size, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05) # оптимизатор параметров методом Adam
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=150) # штука, которая будет уполовинивать (* factor = 1/2) lr при отсутствии улучшений в течение 150 эпох
criterion = nn.BCEWithLogitsLoss() # тот самый лосс, похож на логлосс

In [359]:
# проверяем корректность параметров модели (nn.Linear не нужны)
params_counter = 0
for weights in model.parameters():
    params_counter += weights.shape.numel()
assert params_counter == len(word_to_index) * embedding_dim * 2

In [360]:
def train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    device=device,
):
    pos_labels = torch.ones(batch_size).to(device)
    neg_labels = torch.zeros(batch_size, num_negatives).to(device)
    loss_history = []
    for step in tqdma(range(steps)):
        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs,
            batch_size,
            keep_prob_array,
            word_to_index,
            num_negatives,
            negative_sampling_prob_array,
        ) # сгенерили контекстные пары и neg samples
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(
            device
        ) # ключевое слово - первое в батче
        pos_context_words = torch.tensor(
            [pair[1] for pair in batch], dtype=torch.long
        ).to(device) # положительное - второе
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)
         # негативные генерятся отдельно
        optimizer.zero_grad()
        pos_scores, neg_scores = model(
            center_words, pos_context_words, neg_context_words
        )
        loss_pos = criterion(pos_scores, pos_labels)
        loss_neg = criterion(neg_scores, neg_labels)

        loss = loss_pos + loss_neg
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        lr_scheduler.step(loss_history[-1])

        if step % 100 == 0:
            print(
                f"Step {step}, Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )
    else:
        print(
                f"Resulting Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )


In [361]:
steps = 2500
batch_size = 512
train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
)

  0%|          | 0/2500 [00:00<?, ?it/s]

Step 0, Loss: 4.865993499755859, learning rate: [0.05]
Step 100, Loss: 3.3846240043640137, learning rate: [0.05]
Step 200, Loss: 1.5597336345911026, learning rate: [0.05]
Step 300, Loss: 0.5511732193827629, learning rate: [0.05]
Step 400, Loss: 0.26104101419448855, learning rate: [0.05]
Step 500, Loss: 0.15788865111768247, learning rate: [0.05]
Step 600, Loss: 0.09508352927863598, learning rate: [0.05]
Step 700, Loss: 0.0725910478644073, learning rate: [0.05]
Step 800, Loss: 0.05561760012060404, learning rate: [0.05]
Step 900, Loss: 0.03648724898695946, learning rate: [0.05]
Step 1000, Loss: 0.03202604350633919, learning rate: [0.05]
Step 1100, Loss: 0.025709856173489243, learning rate: [0.05]
Step 1200, Loss: 0.015963920990470795, learning rate: [0.05]
Step 1300, Loss: 0.013495399066014215, learning rate: [0.05]
Step 1400, Loss: 0.015084735385607928, learning rate: [0.05]
Step 1500, Loss: 0.009951074245618656, learning rate: [0.05]
Step 1600, Loss: 0.007282264290261083, learning rate:

Наконец, используйте полученную матрицу весов в качестве матрицы в векторными представлениями слов. Рекомендуем использовать для сдачи матрицу, которая отвечала за слова из контекста (т.е. декодера).

In [362]:
_model_parameters = model.parameters()
embedding_matrix_center = next(
    _model_parameters
).detach()  # Assuming that first matrix was for central word
embedding_matrix_context = next(
    _model_parameters
).detach()  # Assuming that second matrix was for context word

In [363]:
def get_word_vector(word, embedding_matrix, word_to_index=word_to_index):
    return embedding_matrix[word_to_index[word]]

Простые проверки:

In [364]:
similarity_1 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("apple", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("dell", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

In [365]:
similarity_1 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("laptop", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("macbook", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

Наконец, взглянем на ближайшие по косинусной мере слова. Функция реализована ниже.

In [366]:
def find_nearest(word, embedding_matrix, word_to_index=word_to_index, k=10):
    word_vector = get_word_vector(word, embedding_matrix)[None, :]
    dists = F.cosine_similarity(embedding_matrix, word_vector)
    index_sorted = torch.argsort(dists)
    top_k = index_sorted[-k:]
    return [(index_to_word[x], dists[x].item()) for x in top_k.numpy()]

In [367]:
find_nearest("python", embedding_matrix_context, k=10)

[('them', 0.8274371027946472),
 ('topup', 0.829564094543457),
 ('how', 0.8316118121147156),
 ('i', 0.8346576690673828),
 ('part', 0.8374033570289612),
 ('us', 0.8381115198135376),
 ('her', 0.8399948477745056),
 ('harvard', 0.8444579839706421),
 ('water', 0.8486161231994629),
 ('python', 1.0)]

Также вы можете визуально проверить, как представлены в латентном пространстве часто встречающиеся слова.

In [368]:
top_k = 5000
_top_words = sorted([x for x in word_count_dict.items()], key=lambda x: x[1])[
    -top_k - 100 : -100
]  # ignoring 100 most frequent words
top_words = [x[0] for x in _top_words]
del _top_words

In [369]:
word_embeddings = torch.cat(
    [embedding_matrix_context[word_to_index[x]][None, :] for x in top_words], dim=0
).numpy()

In [370]:
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook

output_notebook()


def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs,
):
    """draws an interactive plot for data points with auxilirary info on hover"""
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

Loading BokehJS ...

In [353]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_embeddings)

In [354]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=top_words)

figure(id='p1310', ...)

Для сдачи задания необходимо загрузить функции `subsample_frequent_words` и `get_negative_sampling_prob`, а также сгенерировать файл для посылки ниже и приложить в соответствующую задачу. Успехов!

In [371]:
# do not change the code in the block below
# __________start of block__________
import os
import json

assert os.path.exists(
    "words_subset.txt"
), "Please, download `words_subset.txt` and place it in the working directory"

with open("words_subset.txt") as iofile:
    selected_words = iofile.read().split("\n")


def get_matrix_for_selected_words(selected_words, embedding_matrix, word_to_index):
    word_vectors = []
    for word in selected_words:
        index = word_to_index.get(word, None)
        vector = [0.0] * embedding_matrix.shape[1]
        if index is not None:
            vector = embedding_matrix[index].numpy().tolist()
        word_vectors.append(vector)
    return word_vectors


word_vectors = get_matrix_for_selected_words(
    selected_words, embedding_matrix_context, word_to_index
)

with open("submission_dict.json", "w") as iofile:
    json.dump(word_vectors, iofile)
print("File saved to `submission_dict.json`")
# __________end of block__________

File saved to `submission_dict.json`
